model1

In [ ]:
!git clone https://github.com/cydonia999/VGGFace2-pytorch.git

In [ ]:
%cd VGGFace2-pytorch

In [ ]:
!pip install torch torchvision torchmetrics Pillow

In [5]:
#import 부분

from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import pandas as pd
import os
import json
import random
import pickle
from functools import partial
from models.resnet import resnet50
#정확도 및 mae 임포트
from torchmetrics import Accuracy, MeanAbsoluteError
import numpy as np

In [6]:
class DataProcessing(Dataset):
    '''
    변수 정리

    image_dir :이미지 디렉토리
    label_dir : 라벨 디렉토리 (json형태 리스트형)
    transform : 전처리 및 정규화 (if 문으로 만약 전처리 진행한다면 실행하게..)
    categories : 폴더 감정 4개 정의 되어있는 카테고리 뜻함
    emotion_return : 감정분석까지 할거면 활용할것
    samples : 빈리스트 (채워지는 곳임)

    '''

    def __init__(self, image_dir, label_dir, categories, transform=None, emotion_return=False, mode='train') :
        self.samples=[]
        self.transform = transform
        self.label_map= { cat : idx for idx, cat in enumerate(categories)}
        self.emotion_return = emotion_return
        self.mode = mode
        
        # 나이 정규화 위한 최소/최대 나이 정의 (예: 20~60)
        self.age_min = 10
        self.age_max = 60

        for category in categories :
            json_path =os.path.join(label_dir, f'{self.mode}_{category}.json')
            img_folder = os.path.join(image_dir,category)

            with open(json_path,'r',encoding='euc-kr') as f:
                label_data = json.load(f)

            for row in label_data :
                filename = row['filename']

                img_path = os.path.join(img_folder,filename)

                if os.path.isfile(img_path) :
                    
                    age = row.get('age')
                    
                    # 나이 정규화 (0~1 사이)
                    if age is not None:
                        age_norm = (age - self.age_min) / (self.age_max - self.age_min)
                    else:
                        age_norm = 0.0  # 결측값 처리 예시
                        
                    sample = {
                        'img_path' : img_path,
                        'category' : category,
                        'age' : age_norm,
                        'gender' : row.get('gender')
                    }
                    self.samples.append(sample)


    def __len__(self) :
        return len(self.samples)


    def __getitem__(self,idx) :
        sample = self.samples[idx]
        image = Image.open(sample['img_path']).convert('RGB')

        if self.transform :
            image = self.transform(image)


        #emotion_label = torch.tensor(self.label_map[sample['category']], dtype=torch.long) 수정1
        age=torch.tensor(sample['age'], dtype=torch.float32)
        gender = torch.tensor(1 if sample['gender']=='남' else 0, dtype=torch.long)

        # return image, emotion_label, age, gender 수정1

        if self.emotion_return :
            emotion_label = torch.tensor(self.label_map[sample['category']], dtype=torch.long)
            return image, emotion_label, age, gender

        else :
            return image, age, gender

In [7]:
#transform (개인 커스텀 할 것을 추천드림)
transform = transforms.Compose([
    transforms.Resize((224,224)), #이미지 사이즈 조정
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456,0.406], #RGB평균
                         [0.229,0.224,0.225])  #RGB 표준편차
])

In [ ]:
#데이터 업로드  랜덤으로 합시다 checking용
categories=['anger','happy','panic','sadness']

#상대경로이므로 꼭 맞춰서 다운받을것
base_dir = os.path.dirname(os.path.abspath(__file__))

train_image_dir = os.path.join(base_dir,'Data','img', 'train')
train_label_dir = os.path.join(base_dir,'Data','label', 'train')
val_image_dir = os.path.join(base_dir,'Data','img', 'val')
val_label_dir = os.path.join(base_dir,'Data','label', 'val')

#임의 추출
# def sample_from_dataset(dataset, categories, num_per_category=50) :
#     category_indices={cat: [] for cat in categories}

#     for idx, sample in enumerate(dataset.samples) :
#         category=sample['category']
#         category_indices[category].append(idx)

#     selected_indices = []

#     for cat in categories :
#         indices =category_indices[cat]
#         sampled =random.sample(indices,min(num_per_category, len(indices)))
#         selected_indices.extend(sampled)

#     return selected_indices

# 카테고리별 랜덤 100장씩 추출 (train)
train_data_load=DataProcessing(train_image_dir,train_label_dir,categories,transform=transform, mode='train')
#train_selected_data = sample_from_dataset(train_data_load,categories, num_per_category=200)

#val에서 50장 추출
val_data_load = DataProcessing(val_image_dir, val_label_dir,categories, transform=transform, mode='val')
#val_selected_data = sample_from_dataset(val_data_load, categories, num_per_category=50)


In [9]:
#================= 이부분은 나중에 전체 학습할거라 지워도 무방한곳 확인용으로 만든 코드 =========================================
#train_subset=torch.utils.data.Subset(train_data_load, train_selected_data) #100개 랜덤 뽑기 *4
train_loader= DataLoader(train_data_load, batch_size=32, shuffle=True)

#val_subset=torch.utils.data.Subset(val_data_load, val_selected_data) #50개 랜덤 뽑기*4
val_loader= DataLoader(val_data_load, batch_size=32, shuffle=True)
#========================================================================================================================


In [10]:

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
#=====================모델가져오기================
# model = resnet50()

# # 2. 마지막 fc 레이어 바꾸기 (튜닝용 구조)
# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 256),
#     nn.ReLU(),
#     nn.Dropout(0.3),
#     nn.Linear(256, 3)  # age + gender (2 classes)
# )

In [12]:
# #====================모델 가져오기 2번째 custum v1.1 ==============
# model_v1_1 =resnet50()

# model_v1_1.fc = nn.Sequential(
#     nn.Linear(model_v1_1.fc.in_features,256),
#     nn.ReLU(),
#     nn.Dropout(0.3),
#     nn.Linear(256,128),
#     nn.ReLU(),
#     nn.Dropout(0.2),
#     nn.Linear(128,64),
#     nn.ReLU(),
#     nn.Dropout(0.1),
#     nn.Linear(64,3)
# )

In [13]:
#====================모델 가져오기 2번째 custum v1.2 ==============
model_v1_2_1_1 = resnet50()

model_v1_2_1_1.fc = nn.Sequential(
    nn.Linear(model_v1_2_1_1.fc.in_features,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.4),
    
    nn.Linear(256,128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.3),
    
    nn.Linear(128,64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.2),
    
    nn.Linear(64,3)
)

In [ ]:
#================ 3.가중치 불러오기 ===============
base_dir = os.path.dirname(os.path.abspath(__file__))
weight_path = os.path.join(base_dir, 'resnet50_ft_weight.pkl')

with open(weight_path, 'rb') as f:
    state_dict = pickle.load(f)
    

for key in state_dict:
    if isinstance(state_dict[key], np.ndarray):
        state_dict[key] = torch.from_numpy(state_dict[key])

model_v1_2_1_1.load_state_dict(state_dict, strict=False)


# 5. 디바이스에 올리기
model_v1_2_1_1 = model_v1_2_1_1.to(device)

In [15]:
#================================================
criterion_age = nn.MSELoss()
criterion_gender = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_v1_2_1_1.parameters(), lr=1e-4)
num_epochs = 10 #수정가능

In [16]:
def evaluate(model, data_loader, device, criterion_age, criterion_gender) :
    model.eval()
    total_loss=0
    accuracy = Accuracy(task='binary').to(device)
    mae = MeanAbsoluteError().to(device) 
    
    with torch.no_grad():
        for images, ages, genders in data_loader :
            images = images.to(device)
            ages  = ages.to(device)
            genders = genders.to(device)
            
            outputs = model(images)
            predicted_age = outputs[:,0]
            predicted_gender_logits = outputs[:,1:3]
            
            loss_age = criterion_age(predicted_age, ages)
            loss_gender = criterion_gender(predicted_gender_logits, genders)
            loss = loss_age + loss_gender
            total_loss += loss.item()
            
            pred = torch.argmax(predicted_gender_logits, dim=1)
            accuracy.update(pred, genders)
            mae.update(predicted_age, ages)
    
    avg_loss = total_loss / len(data_loader)        
    return avg_loss, accuracy.compute(), mae.compute()

In [17]:
#=============모델 저장을 위한 빈 리스트 생성=============
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
train_maes = []
val_maes = []

In [18]:
#================Early Stopping======================
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience      # 개선 없을 때 참을 에폭 수
        self.verbose = verbose        # 멈출 때 출력 여부
        self.counter = 0              # 개선 없을 때 카운트
        self.best_loss = np.Inf       # 최저 검증 손실 저장
        self.early_stop = False       # 멈춤 여부
        self.best_model_state = None  # 최적 모델 가중치 저장

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
            if self.verbose:
                print(f'Validation loss improved to {val_loss:.4f}. Saving model.')
        else:
            self.counter += 1
            if self.verbose:
                print(f'No improvement for {self.counter} epochs.')
            if self.counter >= self.patience:
                if self.verbose:
                    print('Early stopping triggered.')
                self.early_stop = True

In [19]:
#===================== 학습 ===========================
early_stopping = EarlyStopping(patience=5, verbose=True)

for epoch in range(num_epochs):
    model_v1_2_1_1.train()
    epoch_loss = 0

    for images, ages, genders in train_loader:
        
        images = images.to(device)
        ages = ages.to(device)
        genders = genders.to(device)

        outputs = model_v1_2_1_1(images)
        predicted_age = outputs[:, 0]
        predicted_gender_logits = outputs[:, 1:3]

        loss_age = criterion_age(predicted_age, ages)
        loss_gender = criterion_gender(predicted_gender_logits, genders)
        loss = loss_age + loss_gender

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    #avg_loss = epoch_loss / len(train_loader)

    # === 평가 ===
    train_loss, train_acc, train_mae = evaluate(model_v1_2_1_1, train_loader, device, criterion_age, criterion_gender)
    val_loss, val_acc, val_mae = evaluate(model_v1_2_1_1, val_loader, device, criterion_age, criterion_gender)

    # === 저장 ===
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc.item())
    val_accuracies.append(val_acc.item())
    train_maes.append(train_mae.item())
    val_maes.append(val_mae.item())

    # ==== 출력 =====
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Train Loss : {train_loss : .4f}, Train Gender Accuracy : {train_acc : .4f}, Train MAE : {train_mae : .4f}')
    print(f'Validation Loss : {val_loss : .4f}, Validation Gender Accuracy : {val_acc : .4f}, Validation MAE : {val_mae : .4f}')
    
    early_stopping(val_loss, model_v1_2_1_1)

    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# 가장 좋은 가중치로 복원
model_v1_2_1_1.load_state_dict(early_stopping.best_model_state)

Epoch [1/10]
Train Loss :  0.3911, Train Gender Accuracy :  0.9161, Train MAE :  0.2760
Validation Loss :  0.4166, Validation Gender Accuracy :  0.8950, Validation MAE :  0.2724
Validation loss improved to 0.4166. Saving model.
Epoch [2/10]
Train Loss :  0.1940, Train Gender Accuracy :  0.9633, Train MAE :  0.1380
Validation Loss :  0.2633, Validation Gender Accuracy :  0.9183, Validation MAE :  0.1364
Validation loss improved to 0.2633. Saving model.
Epoch [3/10]
Train Loss :  0.1324, Train Gender Accuracy :  0.9745, Train MAE :  0.1362
Validation Loss :  0.2275, Validation Gender Accuracy :  0.9275, Validation MAE :  0.1298
Validation loss improved to 0.2275. Saving model.
Epoch [4/10]
Train Loss :  0.1030, Train Gender Accuracy :  0.9862, Train MAE :  0.1312
Validation Loss :  0.2209, Validation Gender Accuracy :  0.9258, Validation MAE :  0.1284
Validation loss improved to 0.2209. Saving model.
Epoch [5/10]
Train Loss :  0.0702, Train Gender Accuracy :  0.9898, Train MAE :  0.1342


<All keys matched successfully>

In [ ]:
base_dir = os.path.dirname(os.path.abspath(__file__))
pth_save_path = os.path.join(base_dir, 'pth_pkl', 'model_raw_weights_v1_2_1_1.pth')

try:
    torch.save(model_v1_2_1_1.state_dict(), pth_save_path)
    print(f'모델 저장 완료 → {pth_save_path}')
except Exception as e:
    print(f'모델 저장 실패: {e}')

In [22]:
history = {
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_accuracies': train_accuracies,
    'val_accuracies': val_accuracies,
    'train_maes': train_maes,
    'val_maes': val_maes
}

In [ ]:
base_dir = os.path.dirname(os.path.abspath(__file__))
pkl_save_path = os.path.join(base_dir, 'pth_pkl', 'model_raw_v1_2_1_1.pkl')

try:
    with open(pkl_save_path, "wb") as f:
        pickle.dump(history, f)
    print(f'학습 기록이 성공적으로 저장되었습니다 : {pkl_save_path}')
except Exception as e:
    print(f'학습 기록 저장 실패: {e}')

In [ ]:
#simple overfit checking 

if train_loss>val_loss or train_acc < val_acc :
    print('overfit예상')

elif train_mae > val_mae :
    print('과소적합 에상')

else:
    print('적절하게 학습되었을 가능성 높음')

정규화 및 Dropout 강화<br>

Dropout 비율 올리기, BatchNorm 층 추가<br>

데이터 증강(Augmentation)<br>

훈련 이미지 다양화<br>

더 많은 데이터 사용 또는 더 적은 복잡도 모델<br>

훈련 데이터가 적다면 데이터 확보<br>

조기 종료(Early Stopping) 도입<br>

검증 성능 안 좋아질 때 학습 중단<br>

학습률 감소 (Learning rate scheduling)<br>

